# TalkingData (Kaggle)
## Pre-processing

### Import

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import train_test_split 
from sklearn.utils import resample

### Load in data

In [2]:
DATA_PATH = r"C:\Users\reio\.kaggle\competitions"

def load_data(data_path=DATA_PATH):
    # PATHS TO FILE
    competition = "talkingdata-adtracking-fraud-detection"
    comp_path = os.path.join(data_path, competition)
    train_sample = os.path.join(comp_path, "train_sample.csv")
    train_path = os.path.join(comp_path, "train.csv")
    test_path = os.path.join(comp_path, "test.csv")
    #ssize = 100000
    #return pd.read_csv(train_path,nrows=ssize), pd.read_csv(test_path)
    return pd.read_csv(train_path), pd.read_csv(test_path)

train, test = load_data()

### Data Exploration

In [ ]:
# Training sample
print(train.shape)
train.head()

In [ ]:
# Describe train
train.dtypes
train.max()

In [ ]:
# Check NAs
train.isnull().sum()

In [ ]:
# Extract data where is_attributed == 1
train_att = train[train['is_attributed']==1]
# Check NAs
train_att.isnull().sum()

We notice that all the missing values in 'attributed_time' are for observations that did not convert into a download ('is_attributed'=0).

In [ ]:
# Percentage of is_attributed == 1
p = len(train_att)/len(train)
print(len(train_att))
print('The percentage of converted clicks is {num:.10%}'.format(num=p))

In [ ]:
# Plot the proportion of clicks that converted into a download or not
plt.figure(figsize=(6,6))
#sns.set(font_scale=1.2)
mean = (train.is_attributed.values == 1).mean()
ax = sns.barplot(['Converted (1)', 'Not Converted (0)'], [mean, 1-mean])
ax.set(ylabel='Proportion', title='Proportion of clicks converted into app downloads')
for p, uniq in zip(ax.patches, [mean, 1-mean]):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height+0.01,
            '{}%'.format(round(uniq * 100, 2)),
            ha="center")

### Undersampling
Sample the data using random undersampling

In [3]:
# Separate the 2 classes
train_0 = train[train['is_attributed'] == 0]
train_1 = train[train['is_attributed'] == 1]

In [4]:
print(len(train_1))
print(train_0.shape)
print(train.shape)
train['is_attributed'].value_counts()

456846
(184447044, 8)
(184903890, 8)


0    184447044
1       456846
Name: is_attributed, dtype: int64

In [5]:
# Undersample class 0 (without replacement)
train0_undersampled = resample(train_0, replace=False, n_samples=len(train_1), random_state=142) 

In [6]:
# Combine minority class with downsampled majority class
train_us = pd.concat([train0_undersampled, train_1])
 
# Display new class counts
train_us.is_attributed.value_counts()

1    456846
0    456846
Name: is_attributed, dtype: int64

### Feature Engineering

In [ ]:
# Set categorical variables
cat = ['ip', 'app', 'device', 'os', 'channel']
for c in cat:
    train_us[c] = train_us[c].astype('category')
    test[c]=test[c].astype('category')

# Only training data has is_attributed
train_us['is_attributed'] = train_us['is_attribute'].astype('category')

In [8]:
# Extract features from click_time
def ppClicktime(df):
    df['click_time'] = pd.to_datetime(df['click_time'])
    df['wday'] = df['click_time'].dt.dayofweek
    df['week'] = df['click_time'].dt.week
    df['hour'] = df['click_time'].dt.hour
    df['minute'] = df['click_time'].dt.minute
    return df
# Pre-process training (undersampled) and testing sets
train_pp = ppClicktime(train_us)
test_pp = ppClicktime(test)

In [9]:
# Drop click_time
train_pp.drop('click_time', axis = 1, inplace = True)
test_pp.drop('click_time', axis = 1, inplace = True)
print(len(test_pp))
test_pp.head()

18790469


,click_id,ip,app,device,os,channel,wday,week,hour,minute
0,0,5744,9,1,3,107,4,45,4,0
1,1,119901,9,1,3,466,4,45,4,0
2,2,72287,21,1,19,128,4,45,4,0
3,3,78477,15,1,13,111,4,45,4,0
4,4,123080,12,1,13,328,4,45,4,0


In [12]:
# Write to csv
train_pp.to_csv("train_pp.csv",index=None)
test_pp.to_csv("test_pp.csv",index=None)

### Feature aggregation

In [39]:
# Load in train_pp
PP_PATH = r"C:\Users\reio\Documents\GitHub\TalkingData"

def load_pp(pp_path=PP_PATH):
    # PATHS TO FILE
    train_pp = os.path.join(pp_path, "train_pp.csv")
    test_pp = os.path.join(pp_path, "test_pp.csv")
    return pd.read_csv(train_pp), pd.read_csv(test_pp)

train_pp, test_pp = load_pp()

In [40]:
# Drop attributed_time
train_pp.drop('attributed_time', axis = 1, inplace = True)

Adding new features

In [41]:
def aggregate_features(df):
    # IPs
    n_ip = df[['ip','channel']].groupby(by=['ip'])[['channel']].count().reset_index().rename(index = str, columns={'channel': 'n_ip'})
    df = df.merge(n_ip, on = ['ip'], how = 'left')
    # wday + hour
    ip_wday_hour = df[['ip', 'wday', 'hour', 'channel']].groupby(by = ['ip','wday','hour'])[['channel']].count().reset_index().rename(index = str, columns = {'channel': 'ip_wday_hour'})
    df = df.merge(ip_wday_hour, on = ['ip', 'wday', 'hour'], how = 'left')
    # app + hour
    ip_app_hour = df[['ip', 'app', 'hour', 'channel']].groupby(by = ['ip','app','hour'])[['channel']].count().reset_index().rename(index = str, columns = {'channel': 'ip_app_hour'})
    df = df.merge(ip_app_hour, on = ['ip', 'app', 'hour'], how = 'left')
    # device + hour
    ip_device_hour = df[['ip', 'device', 'hour', 'channel']].groupby(by = ['ip','device','hour'])[['channel']].count().reset_index().rename(index = str, columns = {'channel': 'ip_device_hour'})
    df = df.merge(ip_device_hour, on = ['ip', 'device', 'hour'], how = 'left')
    # app count
    ip_app_count = df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(columns={'channel': 'ip_app_count'})
    df = df.merge(ip_app_count, on = ['ip', 'app'], how = 'left')
    return df

In [46]:
train_ag = aggregate_features(train_pp)
#aggregate_features(test_pp)

In [47]:
# Write to csv
#train_pp.to_csv("train_ag.csv",index=None)
#test_pp.to_csv("test_ag.csv",index=None)
train_ag.head()

,ip,app,device,os,channel,is_attributed,wday,week,hour,minute,n_ip,ip_wday_hour,ip_app_hour,ip_device_hour,ip_app_count
0,26920,2,1,19,236,0,2,45,4,46,27,1,2,4,4
1,73767,9,1,15,442,0,2,45,4,8,40,2,1,4,5
2,75931,7,2,13,101,0,1,45,6,50,27,2,1,2,1
3,83268,12,1,19,245,0,2,45,6,49,21,1,1,1,5
4,66724,8,1,15,145,0,2,45,0,23,17,1,1,5,1


# Modeling